In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from utils.Config import Config
from model.SpeechLP import SLP
from utils.MLS import MLS

In [8]:
Config.display()

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Text Settings:
  Max Token : 512 tokens

Model Settings:
  Model Name: DiTTO-TTS
  Embedding Dim: 1472
  Num Layers: 1
  Attention Heads: 1

Training Settings:
  Batch Size: 8
  Learning Rate: 0.0001
  Betas: [0.9, 0.999]
  Epochs: 10
  Nb samples: 10000
  Device: cuda


In [9]:
mls = MLS(Config.MAX_TOKEN, nb_samples=Config.NB_SAMPLES, split="train", batch_size=Config.BATCH_SIZE)
batch_loader = mls.loader()

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/34 [00:00<?, ?it/s]

c:\Users\halim\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [10]:
model = SLP(Config.MAX_AUDIO_DURATION, Config.NHEAD ,Config.NUM_LAYERS).to(Config.DEVICE)

In [11]:
import torch.nn as nn
import torch


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), betas=[0.9, 0.999], lr=Config.LEARNING_RATE)
# Imprimer les premiers batches
for e in range(Config.EPOCHS):
    train_loss = 0
    for batch in batch_loader:
        text, audio, labels = batch["text"], batch["audio"].to(Config.DEVICE), batch["labels"].to(Config.DEVICE)
        output = model(text, audio)
        loss = criterion(output, labels)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())
    print(f"[INFO] Epoch {e+1}/{Config.EPOCHS}, Loss : {train_loss/len(batch_loader)}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.